<a href="https://colab.research.google.com/github/Kimchikilla/BostonHousing/blob/main/%EC%BB%B4%ED%93%A8%ED%84%B0%EA%B3%B5%ED%95%99_%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8_%EA%B5%90%EC%96%91%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 교과목 추천 함수
def recommend_courses(semester, major, df, top_n=10):
    # 데이터 전처리: 해당 학기와 전공으로 필터링
    df_filtered = df[(df['학기'] == semester) & (df['전공'] == major) & (df['교과목명'].notna())]

    # 사용자 기반 협업 필터링을 위한 피벗 테이블 생성
    course_matrix = df_filtered.pivot_table(index='학번', columns='교과목명', aggfunc='size').fillna(0)

    # 코사인 유사도 계산
    student_similarity = cosine_similarity(course_matrix)

    # 유사도를 활용한 과목 추천 점수 계산
    similarity_scores = np.mean(student_similarity, axis=1)
    course_scores = course_matrix.T.dot(similarity_scores)
    course_scores = pd.Series(course_scores, index=course_matrix.columns)

    # 상위 N개 과목 추천
    recommended_courses = course_scores.nlargest(top_n)

    # 과목코드와 매칭
    recommended_data = df_filtered[['교과목명', '과목코드']].drop_duplicates().set_index('교과목명')
    recommended_codes = recommended_data.loc[recommended_courses.index, '과목코드']

    return recommended_codes

# 사용자로부터 학기와 학과를 선택하도록 하는 함수
def get_user_input():
    semester = int(input("학기를 입력하세요: "))

    # 파일 로드
    file_path = '/content/drive/MyDrive/24년 2학기 캡스톤 디자인/교양 가상데이터 최종본.csv'
    df_sheet1 = pd.read_csv(file_path)

    # 학과 목록을 불러와 중복 제거하고 NaN 제거
    majors = df_sheet1['전공'].dropna().unique()
    print("\n학과를 선택하세요:")
    for i, major in enumerate(majors, 1):
        print(f"{i}. {major}")

    major_choice = int(input("\n학과 번호를 입력하세요: "))
    selected_major = majors[major_choice - 1]

    # 과목 추천
    recommended_codes = recommend_courses(semester, selected_major, df_sheet1)

    print("\n추천된 상위 10개 교과목의 과목코드:")
    print(recommended_codes.to_list())

# 사용자 데이터 입력 함수 호출
get_user_input()


학기를 입력하세요: 1

학과를 선택하세요:
1. 컴퓨터공학과
2. 도시계획학과
3. 게임소프트웨어학과

학과 번호를 입력하세요: 1

추천된 상위 10개 교과목의 과목코드:
['21995-51', '21995-03', '21995-02', '24699-04', '24699-03', '24699-01', '24699-05', '22434-51', '22434-03', '22434-08', '22434-04', '38455-02', '38455-01', '37610-01', '37610-02', '22377-02', '22377-01', '22377-03', '38465-01', '44667-51', '44667-01', '27274-01', '21272-01', '21272-02']
